
# hw3_CNN

题目：食物的分类

kaggle link:https://www.kaggle.com/c/ml2020spring-hw3 （数据集大小1.1GB）

Strong Baseline: 0.79928

Simple Baseline: 0.70788

第一次提交：0.60251(**<font color='red'>未达标</font>**)

第一次提交：0.79378(**<font color='grenn'>勉强算达标strong baseline吧Q_Q</font>**)

**关键词：** CNN;Classification;

### PyTorch小知识

`torch.utils.data.Dataset`

Dataset分为两种：1. map-style dataset 2. iterable-style dataset

本程序中使用的是map-style dataset

映射形式的数据集的类需要实现\_\_getitem__() 和 \_\_len__()函数来通过index进行数据访问和查看数据集的长度。

For example, such a dataset, when accessed with dataset[idx], could read the idx-th image and its corresponding label from a folder on the disk.


`torch.utils.data.DataLoader` 是pytorch数据加载中最实用的包，能够以迭代的形式加载数据集。

其可以：

* 映射以及迭代的方式加载数据集（map-style and iterable-style）
* 自定义数据加载方式
* 自动分批（batch）
* 单线程或者多线程加载数据
* …………


`torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride = 1, padding = 0, dilation = 1, groups = 1)`

* `in_channels`: 输入通道。 什么是通道：https://blog.csdn.net/sscc_learning/article/details/79814146
* `out_channels`: 输出通道。一般取决于卷积核的数量。
* `kernel_size`: 卷积核的大小，当卷积是方形的时候为整数边长，不是方形要输入元组表示高和宽。
* `stride`: 卷积核移动的步长
* `padding`: 对原图像卷积的填充。
* `dilation`: (先不管，在这里没啥用

### GPU内存不足

  使用了Google的colab，也可以每个月花$9.99购买Google的colab pro https://colab.research.google.com/signup

In [ ]:
!gdown --id '19CzXudqN58R3D-1G8KeFWk8UDQwlb8is' --output food-11.zip # 下載資料集
!unzip food-11.zip # 解壓縮

这里使用谷歌的colab，自己的显卡不太行

In [2]:
import time
import os
import torch as tc
import numpy as np
import pandas as pd
import torch.nn as nn
import torchvision.transforms as transforms
import cv2
from torch.utils.data import dataset, dataloader  # 用于包装data，方便training和testing

引用包文件，这里使用到了dataset和dataloader

In [3]:
# read pics

def read_file(path, label):
    img_dir = sorted(os.listdir(path))
    LEN = len(img_dir) #len(img_dir)
    x = np.zeros((LEN, 128, 128, 3), 'uint8')   # 图片必须是uint8
    y = np.zeros((LEN, ), 'uint8')
    for i, file in enumerate(img_dir):
        if i >= LEN:
            break
        img = cv2.imread(f'{path}/{file}')
        x[i, :, :, :] = cv2.resize(img, (128, 128))  # 调整图片大小
        if label:
            y[i] = int(file.split('_')[0])
    if label:
        return x, y
    else:
        return x


s_time = time.time()
dir_path = './food-11'
train_x, train_y = read_file(os.path.join(dir_path, "training"), True)
valid_x, valid_y = read_file(os.path.join(dir_path, "validation"), True)
# test_x = read_file('food/testing',False)
print(f'read train file over. Time {round(time.time() - s_time, 2)} s')
print(f"Train len: {len(train_x)}")

read train file over. Time 60.0 s
Train len: 9866


读取图片文件。
* 直接创建一个四维数组，用于承载图片，第一位表示图片的数量。记住：因为是图片，因此其np的类型必须是`uint8`的byte类型，否则会报错。
* `cv2.resize()`函数不是截取对应的图片范围，而是将整个图片放缩到对应的像素（会变形）
* 对于不同的系统路径处理使用`os.path.join(dir_path, filename)`处理更好


In [4]:
train_transform = transforms.Compose([  # 一系列处理图片使用Compose
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),  # 随机水平旋转
    transforms.RandomRotation(15),  # 随机旋转
    transforms.ToTensor()  # 图片转为tensor
])

test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

定义transforms, 对图片进行一系列处理，先进行`transforms.ToPILImage()`,最后`transforms.ToTensor()`

In [5]:
class ImgDataset(dataset.Dataset):  # 继承父类dataset
    def __init__(self, x, y=None, transform=None):
        self.x = x
        self.y = y
        if y is not None:
            self.y = tc.LongTensor(y)  # y 应该为Long Tensor
        self.transform = transform

    def __len__(self):  # 实现函数__len__
        return len(self.x)

    def __getitem__(self, index):  # 实现函数__getitem__
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            return X, self.y[index]
        else:
            return X

对于dataset类，需要引入`torch.utils.data`，进行数据处理，dataset和dataloader可以快速方便的对数据集进行数据的迭代选择，方便数据处理。

In [6]:
batch_size = 128
train_set = ImgDataset(train_x, train_y, train_transform) # 顺便进行数据的transform，对于数据进行预处理
valid_set = ImgDataset(valid_x, valid_y, test_transform)
train_loader = dataloader.DataLoader(train_set, batch_size, True) # 设置batch_size对数据进行小批量训练。并且打乱顺序。
valid_loader = dataloader.DataLoader(valid_set, batch_size, False)

设置batch_size和进行data加载器。对于训练集的dataloader要进行打乱顺序进行数据增强，设置为`True`。当然验证集就没有要求了。

In [9]:
# 搭建模型

class Classifier(nn.Module):    # 继承父类nn.Module
    def __init__(self):
        super(Classifier, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # 原图输入维度 (128, 128, 3)
        self.cnn = nn.Sequential(
            # 输入为彩色图片，输出通道64 ，3X3的卷积核，1步长， 填充长度为1=>为了凑128，不然126不容易解决
            nn.Conv2d(3, 64, 3, 1, 1),  # (128, 128, 64)
            nn.BatchNorm2d(64), # norm2d的输入为特征数量，并进行标准化，这里可以设置momentum和eps
            nn.ReLU(),  # 进行ReLU函数转换
            nn.MaxPool2d(2, 2, 0),  # (64, 64, 64)

            # 上一个conv + maxpool的输出为这次的输入
            nn.Conv2d(64, 128, 3, 1, 1),  # (64, 64, 128)
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  # (32, 32, 128)

            nn.Conv2d(128, 256, 3, 1, 1),  # (32, 32, 256)
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  # (16, 16, 256)

            nn.Conv2d(256, 512, 3, 1, 1),  # (16, 16, 512)
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  # (8, 8, 512)

            nn.Conv2d(512, 512, 3, 1, 1),  # (8, 8, 512)
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),  # (4, 4, 512)
        )

        self.fc = nn.Sequential(
            nn.Linear(512 * 4 * 4, 1024),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(1024, 512),
            nn.Dropout(0.5),
            nn.ReLU(),
            nn.Linear(512, 11)
        )

    def forward(self, x):
        out = self.cnn(x) # 进行CNN转换
        out = out.view(out.size()[0], -1)   # 总感觉这句话没什么用
        return self.fc(out) # 输入网络，有四层网络：输入层，隐藏层1，隐藏层2，输出层。最后输出11维度的向量（有11种类别的食品）。

对于分类器一定要继承`nn.Module`。这里进行五步：卷积->标准化->输出->池化。

In [14]:
model = Classifier().cuda()   # 采用GPU模式，构建模型实例
loss = nn.CrossEntropyLoss()  # 因为属于分类，因此定义交叉熵损失函数
# optimizer = tc.optim.Adam(model.parameters(), lr = 0.001, betas=(0.9, 0.999))   # Use adam optimizer, lr是指learning rate
optimizer = tc.optim.SGD(model.parameters(), lr = 0.001, momentum=0.9)
num_epoch = 90

for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = val_acc = 0
    train_loss = val_loss = 0
    model.train()   # 告诉模型开始训练了，启用batch标准化和dropout
    for i, data in enumerate(train_loader):
        optimizer.zero_grad() # 清除原先的导数数据，进行新一轮batch的求导
        train_pred = model(data[0].cuda())
        batch_loss = loss(train_pred, data[1].to('cuda',dtype=tc.long)) # Loss 这里需要做类型转换
        batch_loss.backward() # back propagation
        optimizer.step()  # 设置一个step()

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        # 得到的train_pred每一行是对11中类别的可能行权值，选取每一行中最大的结果的index，argmax返回的是index
        train_loss += batch_loss.item()

    model.eval()
    # 全称 evaluation，不启用batchNormalization和dropout，否则网络的权值会发生改变
    # 使用PyTorch进行训练和测试时一定注意要把实例化的model指定train/eval，eval()时，
    # 框架会自动把BatchNormalization和DropOut固定住，不会取平均，而是用训练好的值，不然的话，一旦test的batch_size过小，
    # 很容易就会被BatchNormalization层导致生成图片颜色失真极大！！！！！！
    with tc.no_grad():  # 停止求导
        for i, data in enumerate(valid_loader):
            val_pred = model(data[0].cuda())
            batch_loss = loss(val_pred, data[1].cuda())  # 注意这里的label需要转为long类型，并且只有GPU和GPU之间的变量才能进行运算

            val_acc += np.sum(np.argmax(val_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
            val_loss += batch_loss.item()

        # 打印结果
        print(f'[{epoch+1}/{num_epoch}]: Time: {round(time.time() - epoch_start_time,2)}s, Train acc/loss:{round(train_acc / len(train_set), 6) * 100}% / {round(train_loss / len(train_set), 6) * 100}%, Validation acc/loss:{round(val_acc / len(valid_set), 6) * 100}% / {round(val_loss / len(valid_set), 6) * 100}%')


[1/30]: Time: 23.84s, Train acc/loss:20.616300000000003% / 1.7656%, Validation acc/loss:25.860100000000003% / 1.6306999999999998%
[2/30]: Time: 24.4s, Train acc/loss:29.201300000000003% / 1.6039%, Validation acc/loss:29.883399999999998% / 1.5143%
[3/30]: Time: 24.7s, Train acc/loss:33.5698% / 1.4997%, Validation acc/loss:37.3469% / 1.3893%
[4/30]: Time: 24.33s, Train acc/loss:36.7423% / 1.4304000000000001%, Validation acc/loss:42.3324% / 1.3163%
[5/30]: Time: 24.28s, Train acc/loss:40.867599999999996% / 1.3501%, Validation acc/loss:41.5452% / 1.3337999999999999%
[6/30]: Time: 24.48s, Train acc/loss:42.377900000000004% / 1.2933%, Validation acc/loss:42.565599999999996% / 1.2991000000000001%
[7/30]: Time: 24.41s, Train acc/loss:46.2903% / 1.2268%, Validation acc/loss:44.344% / 1.2454%
[8/30]: Time: 24.34s, Train acc/loss:47.8613% / 1.1984%, Validation acc/loss:42.1283% / 1.3403%
[9/30]: Time: 24.39s, Train acc/loss:49.6757% / 1.1534%, Validation acc/loss:45.8892% / 1.2833%
[10/30]: Time:

模型构建：分类器构建、损失函数构建、优化器构建
开始训练：
  * 循环每一个epoch
  * 打开train模式，对每一个batch进行训练，BP，更新参数，直至所有数据训练完成
  * 进行交叉验证，打开`eval()`模式，观察train loss和validation loss的变化。一个epoch结束，开始下一个epoch。

In [18]:
# 使用最好的参数
train_val_x = np.concatenate((train_x, valid_x), axis=0)
train_val_y = np.concatenate((train_y, valid_y), axis=0)
train_val_set = ImgDataset(train_val_x, train_val_y, train_transform)
train_val_dataloader = dataloader.DataLoader(train_val_set, batch_size, True)

model_best = Classifier().cuda()
loss = nn.CrossEntropyLoss()
# optimizer = tc.optim.Adam(model_best.parameters(), lr=0.001, betas=(0.9, 0.999))
optimizer = tc.optim.SGD(model_best.parameters(), lr = 0.001, momentum=0.9)
num_epoch = 90
for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc, train_loss = 0, 0

    model_best.train()
    for i, data in enumerate(train_val_dataloader):
        optimizer.zero_grad()
        train_pred = model_best(data[0].cuda())
        batch_loss = loss(train_pred, data[1].cuda())
        batch_loss.backward()
        optimizer.step()

        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(), axis=1) == data[1].numpy())
        train_loss += batch_loss.item()
    print(f'[{epoch+1}/{num_epoch}]: Time: {time.time() - epoch_start_time}, Train acc/loss:{round(train_acc / len(train_val_set), 2) * 100}% / {round(train_loss / len(train_val_set), 2) * 100}%')


[1/90]: Time: 29.24716830253601, Train acc/loss:21.0% / 2.0%
[2/90]: Time: 29.436320304870605, Train acc/loss:31.0% / 2.0%
[3/90]: Time: 29.024319648742676, Train acc/loss:36.0% / 1.0%
[4/90]: Time: 29.201637744903564, Train acc/loss:40.0% / 1.0%
[5/90]: Time: 29.309746265411377, Train acc/loss:44.0% / 1.0%
[6/90]: Time: 29.1187846660614, Train acc/loss:47.0% / 1.0%
[7/90]: Time: 29.155373334884644, Train acc/loss:50.0% / 1.0%
[8/90]: Time: 29.22788166999817, Train acc/loss:52.0% / 1.0%
[9/90]: Time: 29.15738844871521, Train acc/loss:54.0% / 1.0%
[10/90]: Time: 29.185837268829346, Train acc/loss:55.00000000000001% / 1.0%
[11/90]: Time: 29.153873443603516, Train acc/loss:56.99999999999999% / 1.0%
[12/90]: Time: 29.16085648536682, Train acc/loss:59.0% / 1.0%
[13/90]: Time: 29.2264666557312, Train acc/loss:60.0% / 1.0%
[14/90]: Time: 29.234611988067627, Train acc/loss:61.0% / 1.0%
[15/90]: Time: 29.17455506324768, Train acc/loss:62.0% / 1.0%
[16/90]: Time: 29.266878604888916, Train acc/lo

In [19]:
tc.save(model_best,"model_best.torch")  # 保存模型

In [20]:
test_x = read_file(os.path.join(dir_path, 'testing'), False)
test_set = ImgDataset(test_x, transform=test_transform)
test_loader = dataloader.DataLoader(test_set, batch_size, False)
model_best.eval()

predication = []
with tc.no_grad():
    for i, data in enumerate(test_loader):
        test_pred = model_best(data.cuda())
        test_label = np.argmax(test_pred.cpu().data.numpy(), axis=1)
        for y in test_label:
            predication.append(y)

rs = pd.DataFrame(predication,columns=['label'])
rs.to_csv('ans.csv')

print(f'Cost time: {round(time.time() - s_time, 2)} s')

Cost time: 7360.32 s


## 结论与展望
* 第一次照猫画虎，搞清了pytorch的基本使用以及基本流程，得分为0.60251(**<font color='red'>未达标</font>**)

  **分析原因：**
    * 没有设置momentum以及dropout
    * 完整训练的时候数据集使用的是完整测试前train的数据集 :(
* 第二次优化器改用`SGD+momentum`,添加了dropout，获得了0.79378（差一点达到strong 
baseline）

* 说实话感觉还是没有调超参的话，用不着使用validation（evaluation），否则感觉实在浪费时间。两个90个epoch，每个epoch耗费30s，一个小时就这么过去了。
* 看一些大佬的代码，感觉之间的差距实在做CNN的那部分以及神经网络那部分有不同：
  * 除了对于通道 3 -> 64 之间转换之外，其还对自己本身64 -> 64也做了卷积和归一化。并且每一步都做，转到另一个数目的通道时候才池化。
  * 其使用的网络层数比我多一层。

* **这个只是基本了解了pytorch与CNN的流程，后续还需要继续对这一部分进行自我手刻不参考其他人的代码。**

参考文章：

* 理解卷积神经网络中的通道 https://blog.csdn.net/sscc_learning/article/details/79814146 
* torch.nn.Conv2d()的理解 https://blog.csdn.net/qq_38863413/article/details/104108808
* Pytorch中的DataLoader和Dataset https://pytorch.org/docs/stable/data.html
* Pytorch中model.train()与model.eval()的作用 https://blog.csdn.net/qq_38410428/article/details/101102075
* 各种算法收敛比较 https://blog.csdn.net/u010089444/article/details/76725843
* Adam具体介绍以及建议参数 https://blog.csdn.net/leadai/article/details/79178787